In [1]:
import numpy as np
import json, os
from nltk.corpus import wordnet as wn
from math import log
import glob
import tqdm
import json

In [2]:
def _getTextFile(langual):
    file_list = glob.glob(f'../data/stopwords/stopwords/*_{langual}.txt')
    files = ",".join(file_list)
    return files
def countMinMaxAver(lines):
    min_len = 10000
    aver_len = 0
    max_len = 0
    for temp in lines:
        aver_len = aver_len + len(temp)
        if len(temp) < min_len:
            min_len = len(temp)
        if len(temp) > max_len:
            max_len = len(temp)
    aver_len = 1.0 * aver_len / len(lines)
    print('min_len : ' + str(min_len))
    print('max_len : ' + str(max_len))
    print('average_len : ' + str(aver_len))

In [3]:
stop_words=set()
for file in _getTextFile("en").split(","):
    for word in open(file):
        stop_words.add(word.strip())

In [5]:
#sememe_entity_sence = np.load("../data/sememe_entity_sence.npy", allow_pickle=True).tolist()
#title_content_lemmatization_sence = np.load("../data/title_content_lemmatization_sence.npy", allow_pickle=True).tolist()

In [6]:
#title_list = list(title_content_lemmatization_sence.keys())

In [7]:
cdv = np.load("./data/wordnet_voc.npy", allow_pickle=True).tolist()
tagme_sememe_dict_l = np.load("./data/wiki_lem.npy", allow_pickle=True).tolist()

In [10]:
def getCov(data):
    title_sememe_raw = {}
    for k, v in tqdm.tqdm(data.items()):
        sememe = []
        for w in v:
            if w in cdv:
            #if w in dict_sememes:
                sememe.append(w)
        if len(sememe) == 0:
            continue
        title_sememe_raw[k] = sememe

    print("take max [:5000] in content lexion for sememe (raw)")
    countMinMaxAver(title_sememe_raw)
    return title_sememe_raw

In [11]:
tagme_sememe_dict_l = getCov(tagme_sememe_dict_l)

100%|█████████████████████████████| 6329832/6329832 [1:19:22<00:00, 1329.21it/s]


take max [:5000] in content lexion for sememe (raw)
min_len : 1
max_len : 211
average_len : 19.87859688069617


In [12]:
np.save("tagme_sememe_dict_l",tagme_sememe_dict_l)

In [88]:
def cleansememes(tagme_sememe_dict_l):
    sememe_skb = {}
    for k,v in tqdm.tqdm(tagme_sememe_dict_l.items()):
        v = set(v)
        if w in v:
            v.remove(w)
        sememe_skb[k] = list(v)
    return sememe_skb
            

In [89]:
tagme_sememe_dict_l = cleansememes(tagme_sememe_dict_l)

100%|█████████████████████████████| 6115403/6115403 [00:18<00:00, 331511.54it/s]


In [90]:
tagme_sememe_dict_l = {k:v for k,v in tagme_sememe_dict_l.items() if len(v) != 0}

In [91]:
sememe_freq = {}
sememe_set = set()
for doc_sememes in tqdm.tqdm(tagme_sememe_dict_l.values()):
    for sememe in doc_sememes:
        sememe_set.add(sememe)
        if sememe in sememe_freq:
            sememe_freq[sememe] += 1
        else:
            sememe_freq[sememe] = 1

sememe_lexion = list(sememe_set)
sememe_lexion_size = len(sememe_lexion)

sememe_id_map = {}
for i in range(sememe_lexion_size):
    sememe_id_map[sememe_lexion[i]] = i

print("sememe lexion size: ", sememe_lexion_size)

100%|█████████████████████████████| 6115403/6115403 [00:12<00:00, 502398.42it/s]

sememe lexion size:  4455


In [92]:
doc_word_freq = {}
for doc_id,(_, doc_words) in tqdm.tqdm(enumerate(tagme_sememe_dict_l.items())):
    for word in doc_words:
        word_id = sememe_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        if doc_word_str in doc_word_freq:
            doc_word_freq[doc_word_str] += 1
        else:
            doc_word_freq[doc_word_str] = 1
            
word_doc_list = {}
for i,(_, doc_words) in tqdm.tqdm(enumerate(tagme_sememe_dict_l.items())):
    appeared = set()
    for word in doc_words:
        if word in appeared:
            continue
        if word in word_doc_list:
            doc_list = word_doc_list[word]
            doc_list.append(i)
            word_doc_list[word] = doc_list
        else:
            word_doc_list[word] = [i]
        appeared.add(word)
        
word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)
    


6115403it [00:44, 137888.25it/s]
6115403it [00:20, 303068.07it/s]


In [93]:
tfidf_word2doc_all = {}
for i,(title, doc_words) in tqdm.tqdm(enumerate(tagme_sememe_dict_l.items())):
    doc_word_set = set()
    for word in doc_words:
        if word in doc_word_set or word == title:
            continue
        j = sememe_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        idf = log(1.0 * len(tagme_sememe_dict_l) / word_doc_freq[sememe_lexion[j]])
        tfidf_word2doc_all[key] = freq * idf
        doc_word_set.add(word)

6115403it [01:23, 73193.85it/s]


In [37]:
import heapq

In [112]:
def limitSememe(tfidf_word2doc_all,uper,tagme_sememe_dict_l):
    sememe_network = {}
    for title, sememes in tqdm.tqdm(tagme_sememe_dict_l.items()):
        sememes = list(set(sememes))
        if len(sememes) != 0:
            if len(sememes) > uper:
                
                title_index,sememe_index = title.split(",")
                
                limitsememes = []
                score_list = []
                for sememe in sememes:
                    score_list.append(tfidf_word2doc_all[str(title_map[title])+","+str(sememe_id_map[sememe])])
                if len(score_list) == len(sememes):
                    max_index = map(score_list.index, heapq.nlargest(uper,score_list))
                    for max_index_i in set(max_index):
                        limitsememes.append(sememes[max_index_i])
                else:
                    print("ERROR:",title,score_list,sememes)
                
                sememe_network[word] = limitsememes
            
            else:
                sememe_network[word] = sememes
        else:
            continue
    return sememe_network

In [123]:
def limitSememe(tfidf_word2doc_all,uper,tagme_sememe_dict_l):
    sememe_network_index = {}
    
    for title, score in tqdm.tqdm(tfidf_word2doc_all.items()):
        title_index,sememe_index = title.split(",")
        if title_index not in sememe_network_index.keys():
            sememe_network_index[title_index] = []
        sememe_network_index[title_index].append((sememe_index,score))
        
    sememe_network = {}
    title_map = {index:title for index, title in enumerate(tagme_sememe_dict_l.keys())}
    for k, v in tqdm.tqdm(sememe_network_index.items()):
        if len(v) > 10:
            score_list = []
            sememe_list = []
            upper_sememe = []
            for (s_i, sc) in v:
                score_list.append(sc)
                sememe_list.append(sememe_lexion[int(s_i)])
            max_index = map(score_list.index, heapq.nlargest(uper,score_list))
            for max_index_i in max_index:
                upper_sememe.append(sememe_list[max_index_i])
            
            sememe_network[title_map[int(k)]] = upper_sememe
        else:
            sememe_list = []
            for (s_i, _) in v:
                sememe_list.append(sememe_lexion[int(s_i)])
            sememe_network[title_map[int(k)]] = sememe_list
    return sememe_network

In [124]:
sememe_network = limitSememe(tfidf_word2doc_all,10,tagme_sememe_dict_l)

100%|█████████████████████████████| 6114239/6114239 [00:32<00:00, 186310.52it/s]


In [126]:
len(sememe_network)

6114239

In [127]:
word_freq = {}
word_set = set()
for words in sememe_network.values():
    for word in words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)

word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

print("vocab_size: ", vocab_size)

vocab_size:  4455


In [128]:
sememe_network = {k:v for k, v in sememe_network.items() if len(v) != 0}

In [129]:
len(list(sememe_network.values())[0])

9

In [130]:
np.save("./data/wiki_skb",sememe_network)
np.save("./data/wiki_cdv",vocab)